<a href="https://colab.research.google.com/github/walterCNeto/precificador/blob/main/TestesDerivx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install derivx-precificador

In [3]:
from derivx import price_from_spec, bs_call_price

spec = {
  "engine": "mc",
  "model": {"name":"gbm","r":0.05,"q":0.0,"sigma":0.2},
  "grid": {"T": 1.0, "steps": 128},
  "S0": [100.0],
  "product": {"style":"european","type":"european_call","asset":0,"K":100.0},
  "n_paths": 80_000, "seed": 42,
}
p, se = price_from_spec(spec)
print("MC:", p, "±", 1.96*se)
print("BS:", bs_call_price(100, 100, r=0.05, q=0.0, sigma=0.2, T=1.0))

MC: 10.453053775923228 ± 0.10235418620772387
BS: 10.450583572185565


In [4]:
from derivx import price_from_spec, bs_call_price

S0=K=100.0; r=0.05; q=0.0; sigma=0.2; T=1.0
spec = {
  "engine":"mc",
  "model":{"name":"gbm","r":r,"q":q,"sigma":sigma},
  "grid":{"T":T,"steps":128},
  "S0":[S0],
  "product":{"style":"european","type":"european_call","asset":0,"K":K},
  "n_paths":80_000,"seed":42,
}
pmc,se = price_from_spec(spec)
pbs = bs_call_price(S0,K,r,q,sigma,T)
print(f"MC={pmc:.4f} ± {1.96*se:.4f} | BS={pbs:.4f}")

MC=10.4531 ± 0.1024 | BS=10.4506


In [5]:
from derivx import price_from_spec

euro = {"engine":"pde","model":{"name":"gbm","r":0.05,"q":0.0,"sigma":0.2},
        "grid":{"T":1.0},"S0":[100.0],
        "product":{"style":"european","type":"european_put","asset":0,"K":100.0},
        "NS":800,"NT":800,"Smax_mult":5.0}

amer = {**euro, "product":{"style":"american","type":"european_put","asset":0,"K":100.0}}

pe,_ = price_from_spec(euro)
pa,_ = price_from_spec(amer)
print(f"PDE euro={pe:.4f}  PDE amer={pa:.4f}  (amer >= euro)")

PDE euro=5.6138  PDE amer=6.0744  (amer >= euro)


In [6]:
base = {"engine":"mc","model":{"name":"gbm","r":0.05,"q":0.0,"sigma":0.2},
        "grid":{"T":1.0,"steps":128},"S0":[100.0],
        "n_paths":100_000,"seed":7}

van,_   = price_from_spec({**base,"product":{"style":"european","type":"european_call","asset":0,"K":100.0}})
uo130,_ = price_from_spec({**base,"product":{"style":"european","type":"up_and_out_call","asset":0,"K":100.0,"barrier":130.0}})
uo140,_ = price_from_spec({**base,"product":{"style":"european","type":"up_and_out_call","asset":0,"K":100.0,"barrier":140.0}})
print(f"UO130={uo130:.4f} <= UO140={uo140:.4f} <= Vanilla={van:.4f}")

UO130=3.6478 <= UO140=6.0222 <= Vanilla=10.4445


In [7]:
asian,_ = price_from_spec({**base,"product":{"style":"european","type":"asian_arith_call","asset":0,"K":100.0}})
print(f"Asian={asian:.4f} <= Vanilla={van:.4f}")

Asian=5.7958 <= Vanilla=10.4445


In [8]:
def berm(ex_every):
  spec={"engine":"mc","model":{"name":"gbm","r":0.05,"q":0.0,"sigma":0.2},
        "grid":{"T":1.0,"steps":256},"S0":[100.0],
        "product":{"style":"bermudan","type":"european_put","asset":0,"K":100.0,"exercise_every":ex_every},
        "n_paths":120_000,"seed":7}
  return price_from_spec(spec)

for ex in (8,16,32):
  p,se = berm(ex)
  print(f"ex_every={ex:>2}: {p:.4f} ± {1.96*se:.4f}")

ex_every= 8: 6.0395 ± 0.0407
ex_every=16: 6.0427 ± 0.0415
ex_every=32: 6.0361 ± 0.0424


In [9]:
common={"name":"heston","r":0.05,"q":0.0,"kappa":1.5,"theta":0.04,"xi":0.5,"rho":-0.7,"v0":0.04}

fft={"engine":"fft","model":common,"grid":{"T":1.0},"S0":[100.0],
     "product":{"style":"european","type":"european_call","asset":0,"K":100.0},
     "alpha":1.5,"N":4096,"eta":0.25}

mc={"engine":"mc","model":common,"grid":{"T":1.0,"steps":512},"S0":[100.0],
    "product":{"style":"european","type":"european_call","asset":0,"K":100.0},
    "n_paths":200_000,"seed":7}

p_fft,_ = price_from_spec(fft)
p_mc,se = price_from_spec(mc)
print(f"FFT={p_fft:.4f} | MC={p_mc:.4f} ± {1.96*se:.4f}")

FFT=10.4781 | MC=10.4193 ± 0.0643


In [10]:
spec={"engine":"mc",
      "model":{"name":"gbm","r":0.05,"q":[0.01,0.03],"sigma":[0.20,0.30],
               "corr":[[1.0,0.5],[0.5,1.0]]},
      "grid":{"T":2.0,"steps":80},"S0":[100.0,120.0],
      "product":{"style":"european","type":"basket_call","weights":[0.5,0.5],"K":110.0},
      "n_paths":100_000,"seed":3}
p,se = price_from_spec(spec)
print(p, "±", 1.96*se)

15.926254233134596 ± 0.15865480424438438


In [11]:
{
  "engine":"analytic",
  "model":{"r":0.05,"q":[0.0],"sigma":[0.2],"corr":[[1.0]]},
  "grid":{"T":1.0},
  "S0":[100.0],
  "product":{"style":"european","type":"asset_or_nothing_call","asset":0,"K":100.0}
}

{'engine': 'analytic',
 'model': {'r': 0.05, 'q': [0.0], 'sigma': [0.2], 'corr': [[1.0]]},
 'grid': {'T': 1.0},
 'S0': [100.0],
 'product': {'style': 'european',
  'type': 'asset_or_nothing_call',
  'asset': 0,
  'K': 100.0}}

In [12]:
spec = {
  "engine": "analytic",
  "model": {"r": 0.05},   # ou r_curve: {"times":[...], "rates":[...]}
  "grid": {"T": 1.0},
  "S0": [],
  "product": {"style":"european","type":"fra","T1":1.0,"T2":1.5,"tau":0.5,"K":0.045,"notional":1.0}
}
pv,_ = price_from_spec(spec)

In [13]:
pv

0.0026117097297686936

In [14]:
spec = {
  "engine":"analytic","model":{"r":0.05},"grid":{"T":2.0},"S0":[],
  "product":{"style":"european","type":"swap","T0":0.0,
             "payment_times":[0.5,1.0,1.5,2.0],
             "tau":0.5, "fixed_rate":0.05, "notional":1.0}
}
pv,_ = price_from_spec(spec)

In [15]:
pv

0.0011845759417015017

In [16]:
spec = {
  "engine":"analytic","model":{"r":0.05},"grid":{"T":2.0},"S0":[],
  "product":{"style":"european","type":"cap",
             "payment_times":[0.5,1.0,1.5,2.0],
             "tau":0.5,"K":0.04,"sigma":0.20,"notional":1.0}
}
pv,_ = price_from_spec(spec)

In [17]:
pv

0.02072838335158475

In [18]:
spec = {
  "engine":"analytic","model":{"r":0.05},"grid":{"T":2.0},"S0":[],
  "product":{"style":"european","type":"payer_swaption","expiry":1.0,
             "payment_times":[1.5,2.0,2.5],"tau":0.5,"K":0.05,"sigma":0.25,"notional":1.0}
}
pv,_ = price_from_spec(spec)

In [19]:
pv

0.007230983333915667

In [21]:
from pathlib import Path
from derivx.report import plot_report

spec = {
  "engine": "analytic",
  "model": {"r": 0.05, "q": [0.0], "sigma": [0.2], "corr": [[1.0]]},
  "grid": {"T": 1.0},
  "S0": [100.0],
  "product": {"style": "european", "type": "european_call", "asset": 0, "K": 100.0},
}

out_file = "report_call.png"
rep = plot_report(spec, filename=out_file, dpi=150)

print("Preço:", rep.price, "SE:", rep.se, "PNG:", Path(out_file).resolve())

Preço: 10.450583572185565 SE: 0.0 PNG: /content/report_call.png


# Nova seção